In [9]:
import json

exp = 'AKR'

mtg_json = open(exp + '.json', 'r', encoding="utf8")
data = json.load(mtg_json)
mtg_json.close()

data = data['data']
cards = data['cards']

query_pre = '{"bool": {"must": [{"match": {"name.keyword": "'
query_suf = '"}}]}},'

cards_query_ls = set()
for card in cards:
    # 分割カード対応
    if '//' in card['name']:
        sep_pt = card['name'].find(' // ')
        cards_query_ls.add(query_pre + card['name'][:sep_pt] + query_suf)
        cards_query_ls.add(query_pre + card['name'][sep_pt + 4:] + query_suf)
    else:    
        cards_query_ls.add(query_pre + card['name'] + query_suf)
cards_query = "\n".join(cards_query_ls)

# 最後の句は最後の,を削除
cards_query = cards_query[:-1]

with open(exp + '-memo_post_query.txt', 'w', encoding='utf8') as f:
    post_query_pre = 'POST /mtg/_update_by_query?conflicts=proceed&pretty\n{"query": {"bool": {"should": ['
    post_query_suf = ']}},"script": {"inline": """ctx._source.legalities.historic= "Legal";"""}}'
    f.write(post_query_pre + cards_query + post_query_suf)

with open(exp + '-memo_get_query.txt', 'w', encoding='utf8') as f:
    get_query_pre = 'GET /mtg/_search\n{"_source": ["_id","name","legalities"],"size": 1000, "query": {"bool": {"should": ['
    get_query_suf = ']}}}'
    f.write(get_query_pre + cards_query + get_query_suf)